In [1]:
%load_ext autoreload
%autoreload 2
import argparse
import os
import copy
os.environ['http_proxy'] = "http://10.176.52.116:7890"
os.environ['https_proxy'] = "http://10.176.52.116:7890"
os.environ['all_proxy'] = "socks5://10.176.52.116:7891"
import sys
sys.path.append(('../'))
sys.path.append(('../../'))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import json
from typing import Dict
import torch
from torch.nn import CrossEntropyLoss
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from task_vector import TaskVector


from watermarks.kgw.watermark_processor import WatermarkDetector
from watermarks.aar.aar_watermark import AarWatermarkDetector
from watermarks.watermark_types import WatermarkType


In [3]:
from transformers import AutoTokenizer
from datasets import load_dataset
from itertools import chain
from collections import defaultdict, Counter
from transformers import AutoTokenizer

# Parameters based on your provided setup
model_name_or_path = "mistralai/Mistral-7B-v0.1"
dataset_name = "Skylion007/openwebtext"
block_size = 256
per_device_train_batch_size = 8
gradient_accumulation_steps = 4

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

# Load the dataset in streaming mode
dataset = load_dataset(dataset_name, split='train', streaming=True)

# Function to tokenize the text
def tokenize_function(examples):
    return tokenizer(examples["text"])

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Function to decode and calculate n-grams
def calculate_ngrams(sequence, n):
    ngrams = zip(*[sequence[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

# Initialize a dictionary to store all n-grams
unigram_counts = defaultdict(int)
bigram_counts = defaultdict(int)

# Function to group texts into blocks of block_size
def group_texts(examples):
    # Concatenate all texts together
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    
    # Drop the remainder and split into blocks of block_size
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i:i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

# Apply the grouping function to the tokenized dataset
grouped_dataset = tokenized_dataset.map(group_texts, batched=True)

# Function to create batches
def create_batches(dataset, batch_size, gradient_accumulation_steps):
    batch = []
    for example in dataset:
        batch.append(example)
        if len(batch) == batch_size * gradient_accumulation_steps:
            yield batch
            batch = []

# Create batches
batches = create_batches(grouped_dataset, per_device_train_batch_size, gradient_accumulation_steps)
# print(f"Number of batches: {len(list(batches))}")

steps = 2500
# Iterate over the batches and calculate n-grams
progressbar = tqdm(range(steps))
example_number = 0
for batch in batches:
    for example in batch:
        example_number += 1
        decoded_text = tokenizer.decode(example["input_ids"], skip_special_tokens=True)
        unigrams = calculate_ngrams(decoded_text.split(), 1)
        for unigram in unigrams:
            unigram_counts[unigram] += 1
        bigrams = calculate_ngrams(decoded_text.split(), 2)
        for bigram in bigrams:
            bigram_counts[bigram] += 1
    progressbar.update(1)
    steps -= 1
    break
    if steps == 0:
        break

# If you want to convert the defaultdict to a regular dictionary
uningram_counts = dict(unigram_counts)
bigram_counts = dict(bigram_counts)
# Optionally, print out the top n-grams
print(f"There are in total {len(unigram_counts)} unique unigrams and {len(bigram_counts)} unique bigrams.")
print('-'*100)


print("Top 10 most common unigrams:")
for ngram, count in Counter(unigram_counts).most_common(10):
    print(f"{ngram}: {count}")

print("Top 10 most common bigrams:")
for ngram, count in Counter(bigram_counts).most_common(10):
    print(f"{ngram}: {count}")

  0%|          | 0/2500 [00:00<?, ?it/s]2024-08-21 22:10:56.661806: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-21 22:10:58.009216: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 22:10:58.430562: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 22:10:58.619330: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-21 22:10:59.679387: I t

There are in total 2350 unique unigrams and 4914 unique bigrams.
----------------------------------------------------------------------------------------------------
Top 10 most common unigrams:
the: 299
to: 151
a: 125
in: 114
and: 113
of: 109
that: 71
was: 58
his: 47
for: 46
Top 10 most common bigrams:
of the: 30
in the: 24
at the: 16
to the: 15
that the: 10
on the: 10
that he: 9
with the: 9
the American: 9
for the: 8


In [2]:
teacher_model_path = "meta-llama/Llama-2-7b-hf"
teacher_model = AutoModelForCausalLM.from_pretrained(teacher_model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
from watermarks.aar.aar_watermark import AarWatermark
from watermarks.kgw.kgw_watermark import KGWWatermark
from watermarks.kth.kth_watermark import KTHWatermark
from watermarks.watermark_types import WatermarkType

In [31]:
import importlib
import watermarks.kgw.kgw_watermark
from watermarks.kgw.kgw_watermark import KGWWatermark
importlib.reload(watermarks)


<module 'watermarks' (namespace) from ['/remote-home/miintern1/watermark-learnability/experiments/../watermarks', '/remote-home/miintern1/watermark-learnability/watermarks']>

In [32]:
tokenizer = AutoTokenizer.from_pretrained(teacher_model_path)
watermark_configs = {
     "cygu/llama-2-7b-logit-watermark-distill-kgw-k0-gamma0.25-delta1":{"type": "kgw", "k": 0, "gamma": 0.25, "delta": 1.0, "seeding_scheme": "simple_0", "kgw_device": "cpu"},
     "cygu/llama-2-7b-logit-watermark-distill-kgw-k0-gamma0.25-delta2":{"type": "kgw", "k": 0, "gamma": 0.25, "delta": 2.0, "seeding_scheme": "simple_0", "kgw_device": "cpu"},
     "cygu/llama-2-7b-logit-watermark-distill-kgw-k1-gamma0.25-delta1":{"type": "kgw", "k": 1, "gamma": 0.25, "delta": 1.0, "seeding_scheme": "simple_1", "kgw_device": "cpu"},
     "cygu/llama-2-7b-logit-watermark-distill-kgw-k1-gamma0.25-delta2":{"type": "kgw", "k": 1, "gamma": 0.25, "delta": 2.0, "seeding_scheme": "simple_1", "kgw_device": "cpu"},
     "cygu/llama-2-7b-logit-watermark-distill-kgw-k2-gamma0.25-delta2":{"type": "kgw", "k": 2, "gamma": 0.25, "delta": 2.0, "seeding_scheme": "simple_2", "kgw_device": "cpu"},
}
watermark_config = watermark_configs["cygu/llama-2-7b-logit-watermark-distill-kgw-k1-gamma0.25-delta1"]

watermarker = KGWWatermark(
            vocab=tokenizer.get_vocab().values(),
            gamma= watermark_config["gamma"],
            delta=watermark_config["delta"],
            seeding_scheme=watermark_config["seeding_scheme"],
            tokenizer=tokenizer,
            device=watermark_config["kgw_device"],
        )


In [33]:

text = "Tell me how's your day going? I hope you are doing well. I am doing well too."
inputs = tokenizer(text, return_tensors="pt")
print(inputs)
teacher_outputs = teacher_model(**inputs)
# print(teacher_outputs)
print(teacher_outputs.logits.shape)

{'input_ids': tensor([[    1, 24948,   592,   920, 29915, 29879,   596,  2462,  2675, 29973,
           306,  4966,   366,   526,  2599,  1532, 29889,   306,   626,  2599,
          1532,  2086, 29889]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
torch.Size([1, 23, 32000])


In [34]:
logits.shape

torch.Size([1, 23, 32000])

In [35]:
input_ids = inputs["input_ids"]
logits = teacher_outputs.logits
print(logits.shape)
watermark_logits  = watermarker.watermark_logits(input_ids, logits)
print(watermark_logits.shape)
print(torch.all((watermark_logits - logits)==0))

torch.Size([1, 23, 32000])
tensor(20)


IndexError: index 20 is out of bounds for dimension 0 with size 1

In [15]:
hashes = torch.sum(input_ids.unfold(-1, watermarker.k, 1), dim=-1)  # (batch, seq_len - k + 1)
print(hashes.shape)
mask = watermarker.greenlist_masks[hashes]
print(mask.shape)
# logits[..., watermarker.k-1:, :mask.shape[-1]][mask]
print(mask[0][-1])
true_indices = torch.nonzero(mask[0][-1], as_tuple=False).squeeze()
print(true_indices[0])

torch.Size([1, 23])
torch.Size([1, 23, 32000])
tensor([False, False, False,  ..., False,  True, False])
tensor(20)


In [20]:
test = mask.view(-1)
test.shape, mask.shape

(torch.Size([736000]), torch.Size([1, 23, 32000]))